# tubelet-service gRPC client reference/tutorial

Before running this notebook please make sure to run the service. 

In [13]:
SERVICE_ENDPOINT = 'localhost:50051'

DEFAULT_WINDOW_WIDTH = 320  # FIXME: should be configurable in service
DEFAULT_WINDOW_HEIGHT = 200 # FIXME: should be configurable in service
CLUSTERING_DISTANCE = 3.0 # FIXME: should be configurable in service

### Client Initialization

In [14]:
import grpc

from tubelet_service_pb2_grpc import tubelet_serviceStub
from tubelet_service_pb2 import Empty

channel = grpc.insecure_channel(SERVICE_ENDPOINT)
tubc = tubelet_serviceStub(channel)

### get_version()

In [15]:
response = tubc.get_version(request=Empty())
print("Version:",response.version)

Version: 0.5.0


### reset()

Deletes all tubelets.

In [16]:
response = tubc.reset(request=Empty())

### add_observation2d()

Adds an observation at a given point. If that point already contains a tubelet, the observation is added to the existing tubelets.

In [17]:
from tubelet_service_pb2 import add_observation2d_request, point2d, observation

observations = [
    # The following points will be clustered in the same tubelet    
    ( 10.0, 30.0, 1, 0.11 ),
    ( 11.0, 30.4, 2, 0.111 ),
    ( 10.0, 29.97, 3, 0.21 ),
    ( 10.5, 31.0, 4, 0.81 ),
    ( 9.8,  30.6, 5, 0.94 ),
    
    # The following points will be stored in new tubelets
    ( 40.0, 19.97, 10, 0.01 ),
    ( 60.5, 21.0, 11, 0.51 ),
    ( 70.8,  30.6, 12, 0.64 )
]

response = tubc.reset(request=Empty())

for obs in observations:
    print("Adding point (%f,%f) of class %d with prob=%f" % (obs[0], obs[1], obs[2], obs[3]) ) 
    response = tubc.add_observation2d(
        request=add_observation2d_request(
            pos = point2d(x=obs[0],y=obs[1]),
            obs = observation(class_index=obs[2],prob=obs[3])
        )
    )

Adding point (10.000000,30.000000) of class 1 with prob=0.110000
Adding point (11.000000,30.400000) of class 2 with prob=0.111000
Adding point (10.000000,29.970000) of class 3 with prob=0.210000
Adding point (10.500000,31.000000) of class 4 with prob=0.810000
Adding point (9.800000,30.600000) of class 5 with prob=0.940000
Adding point (40.000000,19.970000) of class 10 with prob=0.010000
Adding point (60.500000,21.000000) of class 11 with prob=0.510000
Adding point (70.800000,30.600000) of class 12 with prob=0.640000


### get_all_tubelets2d()

Obtains all the tubelets in the tree for the current window

In [18]:
def print_all_tubelets():
    response = tubc.get_all_tubelets2d(request=Empty())
    index = 0
    if len(response.tubelets)>0:
        for t in response.tubelets:    
            print("Tubelet %d" % index, t)
            index+=1
            
print_all_tubelets()            

Tubelet 0 pos {
  x: 10.0
  y: 30.0
}
observations {
  class_index: 1
  prob: 0.11
}
observations {
  class_index: 2
  prob: 0.111
}
observations {
  class_index: 3
  prob: 0.21
}
observations {
  class_index: 4
  prob: 0.81
}
observations {
  class_index: 5
  prob: 0.94
}

Tubelet 1 pos {
  x: 40.0
  y: 19.97
}
observations {
  class_index: 10
  prob: 0.01
}

Tubelet 2 pos {
  x: 60.5
  y: 21.0
}
observations {
  class_index: 11
  prob: 0.51
}

Tubelet 3 pos {
  x: 70.8
  y: 30.6
}
observations {
  class_index: 12
  prob: 0.64
}



# Moving and clipping example

All tubelets outside the viewing window are deleted with clip() function.
Window starts at (0,0). After moving (dx=40,dy=0) all tubelets outside the new region: 

(x0=40,y0=0,x1=40+DEFAULT_WINDOW_WIDTH,y1=y0+DEFAULT_WINDOW_HEIGHT) 

will be deleted.

In [19]:
from tubelet_service_pb2 import move_window_request

observations = [
    # The following points will be clustered in the same tubelet    
    ( 10.0, 30.0, 1, 0.11 ),
    ( 11.0, 30.4, 2, 0.111 ),
    ( 10.0, 29.97, 3, 0.21 ),
    ( 10.5, 31.0, 4, 0.81 ),
    ( 9.8,  30.6, 5, 0.94 ),
    
    # The following points will be stored in new tubelets
    ( 40.0, 19.97, 10, 0.01 ),
    ( 60.5, 21.0, 11, 0.51 ),
    ( 70.8,  30.6, 12, 0.64 )
]

# Reset
response = tubc.reset(request=Empty())

# Add observations
for obs in observations:
    print("Adding point (%f,%f) of class %d with prob=%f" % (obs[0], obs[1], obs[2], obs[3]) ) 
    response = tubc.add_observation2d(
        request=add_observation2d_request(
            pos = point2d(x=obs[0],y=obs[1]),
            obs = observation(class_index=obs[2],prob=obs[3])
        )
    )

Adding point (10.000000,30.000000) of class 1 with prob=0.110000
Adding point (11.000000,30.400000) of class 2 with prob=0.111000
Adding point (10.000000,29.970000) of class 3 with prob=0.210000
Adding point (10.500000,31.000000) of class 4 with prob=0.810000
Adding point (9.800000,30.600000) of class 5 with prob=0.940000
Adding point (40.000000,19.970000) of class 10 with prob=0.010000
Adding point (60.500000,21.000000) of class 11 with prob=0.510000
Adding point (70.800000,30.600000) of class 12 with prob=0.640000


In [20]:
print_all_tubelets()   

Tubelet 0 pos {
  x: 10.0
  y: 30.0
}
observations {
  class_index: 1
  prob: 0.11
}
observations {
  class_index: 2
  prob: 0.111
}
observations {
  class_index: 3
  prob: 0.21
}
observations {
  class_index: 4
  prob: 0.81
}
observations {
  class_index: 5
  prob: 0.94
}

Tubelet 1 pos {
  x: 40.0
  y: 19.97
}
observations {
  class_index: 10
  prob: 0.01
}

Tubelet 2 pos {
  x: 60.5
  y: 21.0
}
observations {
  class_index: 11
  prob: 0.51
}

Tubelet 3 pos {
  x: 70.8
  y: 30.6
}
observations {
  class_index: 12
  prob: 0.64
}



In [21]:
# Move and clip
response = tubc.move_window(request=move_window_request(dx=40.0,dy=0.0))
response = tubc.clip(request=Empty()) #FIXME

In [22]:
print_all_tubelets()   

### Client deinitialization

In [23]:
channel.close()